In [33]:
import pandas as pd
import numpy as np

2019のデータらしい

In [34]:
# aggregateのところだけを抜き出す、idは別で保存
data_3=pd.read_csv('igarashi2.csv')
main_ids=data_3["分類"]
data_3=data_3.iloc[[4,7,8,17,18,19],[1,3,4,5]]
data_3.reset_index(drop=True,inplace=True)
data_3
data_3["otc"]=data_3["otc"].astype(int)
data_3["税制対象"]=data_3["税制対象"].astype(int)
data_3["otc_per"]=data_3["otc"]/data_3.iloc[5,1]
data_3["sotc_per"]=data_3["税制対象"]/data_3.iloc[5,2]
data_3["薬効"]
data_3

,薬効,otc,税制対象,税制対象割合,otc_per,sotc_per
0,かぜ関連小計,161468727,77904872,48.2%,0.369691,0.558973
1,鼻炎関連小計,31626317,15525556,49.1%,0.072410,0.111397
2,鎮痛・鎮痒・収斂・消炎剤,91018396,34534873,37.9%,0.208391,0.247790
3,消化器官用薬合計,72207173,5816576,8.1%,0.165322,0.041734
4,ビタミン剤,80445821,5589554,6.9%,0.184185,0.040105
5,合計,436766434,139371431,31.9%,1.000000,1.000000


In [35]:
data_ndb=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
data_ndb=data_ndb[data_ndb["year"]==2019]
data_ndb.shape

(3493, 112)

In [36]:
main_ids.dropna(inplace=True)
main_ids
unique_items = []
for item in main_ids:
    item_data = item.split()
    unique_items.extend(item_data)

idset=set(np.array(unique_items).astype(int))
print(idset)
np.array(unique_items).astype(int)
ids=[]
ids.append([114, 118, 222, 223, 224, 441])
ids.append([132, 449])
ids.append([264])
ids.append([231, 232, 233, 234,235, 236, 237, 239])
# ids
prices=[]

{224, 449, 132, 231, 264, 232, 233, 234, 235, 236, 237, 239, 114, 118, 441, 31, 222, 223}


In [37]:
for i in range(4):
    prices.append(data_ndb[data_ndb["薬効分類"].isin(ids[i])]["薬価"].mean())
prices

[61.29607427055703, 335.01701030927836, 15.068, 59.86833984374999]

In [38]:
data_3["otc_per"].sum()-1,data_3["sotc_per"].sum()-1

(1.0, 1.0)

全項目じゃないほう: 初再診+医学管理等＋投薬＋調剤+薬剤

0,6,7,9,11 #解熱鎮痛剤 全部同じ  
1,2 #鼻炎 同じ  
3,4,5 #胃腸 4,5は同じ  
8,10,12 #外用消炎　全部同じ  

In [39]:
data_price=pd.read_csv('igarashi.csv')
data_price["金額/日"]

0     173.570
1     140.465
2     140.465
3      27.390
4      75.208
5      75.208
6     107.422
7     107.422
8      15.612
9     107.422
10     15.612
11    107.422
12     15.612
Name: 金額/日, dtype: float64

In [40]:
data_price=data_price.drop_duplicates(subset='薬効',keep='first')
data_price["rx金額/日"]=data_price["3割負担"]/data_price["平均日数（枚数）"]
data_price.reset_index(drop=True,inplace=True)
data_price

,症状,レセプト件数,全項目金額,全項目金額/件,全項目3割負担,金額,金額/件,3割負担,平均日数（枚数）,薬効,金額/日,OTC金額,rx金額/日
0,かぜ （全薬剤）,5379,33152340,6163,1849,30199740,5614,1684,4.889,かぜ薬,173.570,849,344.446717
1,鼻炎① （全薬剤）,77598,667322650,8600,2580,572286677,7375,2213,28.200,抗ア薬,140.465,3961,78.475177
2,便秘 （全薬剤）,4134,24054390,5819,1746,21592212,5223,1567,32.168,便秘薬,27.390,881,48.713007
3,胃炎 （全薬剤）,635,3648900,5746,1724,3092507,4870,1461,7.062,胃腸薬,75.208,531,206.881903
4,頭痛 （全薬剤）,1495,7576520,5068,1520,6138331,4106,1232,6.153,解熱鎮痛,107.422,661,200.227531
5,腰痛② （外用消炎）,593,4319520,7284,2185,2633273,4441,1332,81.430,外用消炎,15.612,1271,16.357608


share otc  
0 かぜ関連小計  
1          鼻炎関連小計  
2    鎮痛・鎮痒・収斂・消炎剤  
3        消化器官用薬合計  
4           ビタミン剤  
5              合計

In [41]:
# 重みづけOTC価格
# pay per day
ppd=list(data_price.loc[:,"金額/日"].astype(int))
ppd_rx=list(data_price.loc[:,"rx金額/日"].astype(int))
# sotc全体に対するシェアで重みづける、これが重み
share_sotc=list(data_3.iloc[:,4])
ppd[0]*share_sotc[0]+ppd[1]*share_sotc[1]+ppd[2]*share_sotc[3]/3+ppd[3]*share_sotc[3]*2/3+ppd[4]*share_sotc[2]+ppd[5]*share_sotc[2]
# ppd[0]*share_sotc[0]+ppd[1]*share_sotc[1]+ppd[2]*share_sotc[3]/2+ppd[3]*share_sotc[3]*1/2+ppd[4]*share_sotc[2]+ppd[5]*share_sotc[2]

109.27177080187441

In [42]:
# 重みづけ薬価
avg_pr=0
for i in range(4):
    avg_pr+=prices[i]*share_sotc[i]
avg_pr

59.95685559995722

In [21]:
# 重みづけ３割負担
ppd=ppd_rx
ppd[0]*share_sotc[0]+ppd[1]*share_sotc[1]+ppd[2]*share_sotc[3]/3+ppd[3]*share_sotc[3]*2/3+ppd[4]*share_sotc[2]+ppd[5]*share_sotc[2]

203.18373384129302

In [1]:
203/60

3.3833333333333333

そもそもセルメ主要薬効はどれくらいを占めるのか

In [22]:
# data ship
data_ship=pd.read_csv("../merged/len3_ship_DN.csv",encoding="shift-jis",index_col=0)
data_ship=data_ship[data_ship["year"]==2019]
# data_s["id"].astype(str)

data_ship.head()


,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
275,2019,114,解熱鎮痛消炎剤,143805120,55891566.0,37376219,37427896.0,10323037.0,7378055.0,16911795.0,...,0,0,0,0,0,0,0,0,0,0
276,2019,121,局所麻酔剤,11455407,0.0,0,2721089.0,0.0,0.0,995778.0,...,0,0,0,0,0,0,0,0,0,0
277,2019,123,自律神経剤,2340838,0.0,0,443724.0,0.0,0.0,89317.0,...,0,0,0,0,0,0,0,0,0,0
278,2019,131,眼科用剤,317582854,71689734.0,3927990,35943525.0,11120794.0,2159959.0,76672849.0,...,0,0,0,0,0,0,0,0,0,0
279,2019,132,耳鼻科用剤,41068867,20513544.0,4429304,17291231.0,5797883.0,2384657.0,3896325.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
data_ship_main=data_ship[data_ship["id"].isin(idset)]
data_ship_main["self_dom"].sum()/data_ship["self_dom"].sum(),data_ship_main["otc_dom"].sum()/data_ship["otc_dom"].sum()

(0.8342919807740469, 0.6606591447997165)

rxとの比

In [24]:

data_price["金額/日2"]=data_price["金額/日"].copy()
data_price.loc[9,"金額/日2"]=data_price["金額/日"][9]+data_price["金額/日"][10]
data_price.loc[11,"金額/日2"]=data_price.loc[11,"金額/日"]+data_price.loc[12,"金額/日"]
data_price["金額/日"]/data_price["rx金額/日"],np.mean(data_price["金額/日"]/data_price["rx金額/日"])

KeyError: 9

RXの金額で重みづけるか

In [ ]:
data_2=data.copy()
data_2.drop([10,12],inplace=True)
data_2.reset_index(drop=True,inplace=True)
data_2

,症状,レセプト件数,全項目金額,全項目金額/件,全項目3割負担,金額,金額/件,3割負担,平均日数（枚数）,薬効,金額/日,OTC金額,rx金額/日,金額/日2
0,かぜ （全薬剤）,5379,33152340,6163,1849,30199740,5614,1684,4.889,かぜ薬,173.570,849,344.446717,173.570
1,鼻炎① （全薬剤）,77598,667322650,8600,2580,572286677,7375,2213,28.200,抗ア薬,140.465,3961,78.475177,140.465
2,鼻炎② （抗ア薬）,37709,292673860,7761,2328,250699601,6648,1994,28.686,抗ア薬,140.465,4029,69.511260,140.465
3,便秘 （全薬剤）,4134,24054390,5819,1746,21592212,5223,1567,32.168,便秘薬,27.390,881,48.713007,27.390
4,胃炎 （全薬剤）,635,3648900,5746,1724,3092507,4870,1461,7.062,胃腸薬,75.208,531,206.881903,75.208
5,胸やけ （全薬剤）,1402,11153680,7956,2387,9031581,6442,1933,29.344,胃腸薬,75.208,2207,65.873773,75.208
6,頭痛 （全薬剤）,1495,7576520,5068,1520,6138331,4106,1232,6.153,解熱鎮痛,107.422,661,200.227531,107.422
7,腰痛① （消炎鎮痛）,118,661040,5602,1681,495376,4198,1259,9.933,解熱鎮痛,107.422,1067,126.749220,107.422
8,腰痛② （外用消炎）,593,4319520,7284,2185,2633273,4441,1332,81.430,外用消炎,15.612,1271,16.357608,15.612
9,腰痛③,833,6069780,7287,2186,3859760,4634,1390,10.110,解熱鎮痛,107.422,1998,137.487636,123.034


In [ ]:
sum_p=data_2["金額"].sum()
sum_p

911628865

In [ ]:
data_2["金額/日2"]/data_2["rx金額/日"]

0     0.503910
1     1.789929
2     2.020752
3     0.562273
4     0.363531
5     1.141699
6     0.536500
7     0.847516
8     0.954418
9     0.894873
10    1.008913
dtype: float64

In [ ]:
# 平均的にrxの何倍の価格か
np.sum((data_2["金額/日2"]/data_2["rx金額/日"])*(data_2["金額"]/sum_p))

1.7453724590201782

In [ ]:
# 平均的に何日分か、平均金額は何円か
np.sum(data_2["平均日数（枚数）"]*(data_2["金額"]/sum_p)),np.sum(data_2["金額/件"]*(data_2["金額"]/sum_p))

(27.305563298647854, 6971.556509985015)